In [1]:
import re
import json
import numpy as np

In [109]:
month2number = {
    'января': '01',
    'февраля': '02',
    'марта': '03',
    'апреля': '04',
    'мая': '05',
    'июня': '06',
    'июля': '07',
    'августа': '08',
    'сентября': '09',
    'октября': '10',
    'ноября': '11',
    'декабря': '12'
}

reg_exprs = [
    re.compile(r'от (\d?\d) ([а-я]+) (\d\d\d\d)'),
    re.compile(r'(\d\d)\.(\d\d)\.(\d\d\d\d)'),
    re.compile(r'\n(\d?\d) ([а-я]+) (\d\d\d\d)')
]

In [267]:
def process_match(match):
    day = match[0] if len(match[0]) == 2 else '0' + match[0]
    month = month2number[match[1]] if match[1] in month2number else match[1]
    year = match[2]
    return (day, month, year)

def extract_date(doc, doc_type):
    matches = []
    variants = []
    doc = '\n' + doc.lower() + '\n'
    doc = re.sub(r'\n+', r'\n', doc)
    
    if doc_type == 'федеральный закон':
        match = re.search(r'\n(\d?\d) ([а-я]+) (\d\d\d\d) года\n', doc)
        if match:
            date = process_match((match.group(1), match.group(2), match.group(3)))
            return f'{date[0]}.{date[1]}.{date[2]}'
        
    if doc_type == 'приказ':
        match = re.search(r'приказ\n([^\n]*\n)?([^\n]*\n)?(от )?(\d?\d) ([а-я]+) (\d\d\d\d)', doc)
        if match:
            date = process_match((match.group(4), match.group(5), match.group(6)))
            return f'{date[0]}.{date[1]}.{date[2]}'
        match = re.search(r'приказ\n([^\n]*\n)?([^\n]*\n)?(от )?(\d\d\.\d\d\.\d\d\d\d)', doc)
        if match:
            return match.group(4)
            
    if doc_type == 'указ':
        match = re.search(r'указ\n([^\n]*\n)?([^\n]*\n)?([^\n]*\n)?(от )?(\d?\d) ([а-я]+) (\d\d\d\d)', doc)
        if match:
            date = process_match((match.group(5), match.group(6), match.group(7)))
            return f'{date[0]}.{date[1]}.{date[2]}'
        match = re.search(r'указ\n([^\n]*\n)?([^\n]*\n)?([^\n]*\n)?(от )?(\d\d\.\d\d\.\d\d\d\d)', doc)
        if match:
            return match.group(5)
            
    if doc_type == 'распоряжение':
        match = re.search(r'\n(\d\d\.\d\d\.\d\d\d\d)[^\n]*\n№ [\w\-/а-я]+\n', doc)
        if match:
            return match.group(1)
    
    if doc_type == 'постановление':
        match = re.search(r'постановление\n([^\n\d]*\n)?([^\n\d]*\n)?([^\n\d]*\n)?(от[_\- ]+?)?(\d?\d) ([а-я]+) (\d\d\d\d)', doc)
        if match:
            date = process_match((match.group(5), match.group(6), match.group(7)))
            return f'{date[0]}.{date[1]}.{date[2]}'
        match = re.search(r'постановление\n([^\n\d]*\n)?([^\n\d]*\n)?([^\n\d]*\n)?[а-я_\- ]*(\d\d\.\d\d\.\d\d\d\d)', doc)
        if match:
            return match.group(4)
            
    if doc_type == 'закон':
        match = re.search(r'\n(\d\d\.\d\d\.\d\d\d\d)[^\n]*\n№ [\w\-/а-я]+\n', doc)
        if match:
            return match.group(1)
        match = re.search(r'\n(от )?(\d?\d) ([а-я]+) (\d\d\d\d)[^\n]*\n№ [\w\-/а-я]+\n', doc)
        if match:
            date = process_match((match.group(2), match.group(3), match.group(4)))
            return f'{date[0]}.{date[1]}.{date[2]}'
    
    for expr in reg_exprs:
        matches += expr.findall(doc)
    if matches:
        for match in matches:
            variants.append(process_match(match))
    else:
        matches = re.findall(r'(\d?\d) ([а-я]+) (\d\d\d\d)', doc)
        for match in matches:
            variants.append(process_match(match))
            
    date = variants[0] if variants else ''
    for variant in variants:
        if f'{variant[2]}.{variant[1]}.{variant[0]}' > f'{date[2]}.{date[1]}.{date[0]}':
            date = variant
    return f'{date[0]}.{date[1]}.{date[2]}' if date else date

def predict(test, types):
    results = []
    for doc, doc_type in zip(test, types):
        date = extract_date(doc, doc_type)
        prediction = {"type": "",
                      "date": date,
                      "number": "",
                      "authority": "",
                      "name": ""}
        results.append(prediction)
    return results

In [268]:
from eval_module import quality

train = []
labeled = []
names = []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        names.append(docname)
        with open(docname, 'r') as f:
            train.append(f.read())
        labeled.append(doc_dict['label'])

true_types = np.array([x['type'] for x in labeled])
predicted = predict(train, true_types)
quality(predicted, labeled)

{'date_accuracy': 0.9889696666,
 'number_accuracy': 0.0,
 'type_f1_score': 0.0,
 'name_jaccard': 0.0,
 'authority_jaccard': 0.0,
 'subtasks_improves': 1}

In [269]:
predicted_labels = np.array(predicted)
test_labels = np.array(labeled)
names = np.array(names)
for doc_type in ['федеральный закон', 'постановление', 'приказ', 'распоряжение','закон', 'указ']:
    ids = np.where(true_types == doc_type)
    print(f"{doc_type}: {quality(predicted_labels[ids], test_labels[ids])['date_accuracy']}")

федеральный закон: 1.0
постановление: 0.9890199268
приказ: 0.974025974
распоряжение: 1.0
закон: 0.9849246231
указ: 0.9810126582


In [270]:
doc_type = 'закон'
ids = np.where(true_types == doc_type)

y_test = [(x['date'], x['type']) for x in test_labels[ids]]
y_pred = [x['date'] for x in predicted_labels[ids]]

wrong_docs = []
i = 0
for test, pred in zip(y_test, y_pred):
    if test[0] != pred:
        wrong_docs.append((names[ids][i], test, pred))
    i += 1
    
for doc_name, test, pred in wrong_docs:
    with open(doc_name, 'r') as f:
        print(f"right value = {test}")
        print(f"wrong value = {pred}")
        print(f.read())
        print("=====")

right value = ('10.12.2014', 'закон')
wrong value = 25.06.2012
ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ -- ЮГРА

ЗАКОН

©О внесении изменений в отдельные законы
Ханты-Мансийского автономного округа - Югры

Принят Думой Ханты-Мансийского
автономного округа - Югры 10 декабря 2014 года

Статья 1. Внести в Закон Ханты-Мансийского автономного округа -
Югры от 10 июля 2009 года № 109-о3 "О мерах по реализации отдельных положений Федерального закона "Об основных гарантиях прав ребенка в Российской Федерании" в Ханты-Мансийском автономном округе - Юргре" (с изменениями, внесенными законами Ханты-Мансийского автономного округа - Югры
от 17 дёкабря 2009 года № 226-оз, 16 декабря 2010 года № 239-оз, 30 сентября
2013 года № 72-оз, 7 ноября 2013 года № 108-оз) (Собрание законодательства
Ханты-Мансийского автономного округа - Югры, 2009, № 7 (ч. 1), ст. 585;
№ 12 (с.), ст. 1169; 2010, № 12 (ч. 3), ст. 1157; 2013, № 9 (ч. 2, т. |), ст. 1110;
2013, № 11 (ч. 1), ст. 1321) следующие изменения:

1. В подпунктах

# Для каждого типа документа свой извлекатор:

* указ - дата в конце 12 июня 2012 года

* постановление - дата в начале (от) 17.12.2015 г., 10 марта 2015 года 

* распоряжение - дата в начале от 27 ноября 2014 г.
